In [ ]:
import os
import json
from openai import OpenAI

client = OpenAI(api_key="Please paste your API key here")

def read_file_content(file_path: str) -> str:
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()


def evaluate_dmn_with_prompt(dmn_content: str, case_text: str) -> dict:
    """
    General-purpose evaluation of ALL rules in a DMN file against case text.
    Returns JSON with results for each rule.
    """
    prompt = """
You are given two inputs:
1. A DMN file in XML format (may contain multiple decision tables and literal expressions).
2. A natural language case description.

Your task:
- Extract ALL rules from the DMN (use annotations or IDs as names).
- For each rule, evaluate whether the case text satisfies the rule expression.
- If satisfied, mark "RuleX:true".
- If not satisfied, mark "RuleX:false" and append the message from the DMN.

⚠️ Important:
- Respect interval notations ([40..50], (40..50), etc.).
- Do not approximate values (e.g., 39 does not match [40..50]).
- Return strictly valid JSON in this format:

{
  "result": [
    "Rule1:true",
    "Rule2:false\\n<Message>",
    "Rule3:true"
  ]
}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a DMN expert. Always return valid JSON."},
            {"role": "user", "content": prompt +
                "\n\n--- DMN FILE START ---\n" + dmn_content + "\n--- DMN FILE END ---" +
                "\n\n--- CASE TEXT START ---\n" + case_text + "\n--- CASE TEXT END ---"
            }
        ],
        temperature=0,
        response_format={"type": "json_object"}  # if supported by your SDK
    )

    return json.loads(response.choices[0].message.content)


def evaluate_dmn(dmn_path: str, case_path: str):
    """
    Orchestrates full evaluation with file I/O.
    """
    dmn_content = read_file_content(r"Locate your DMN file")
    case_text = read_file_content(r"Locate your text file")

    return evaluate_dmn_with_prompt(dmn_content, case_text)


if __name__ == "__main__":
    dmn_file = r"Locate to your DMN file"
    case_file = r"Locate to your text file"

    results = evaluate_dmn(dmn_file, case_file)

    print("\n✅ Final Evaluation Results:")
    for r in results["result"]:
        print(r)
